# Tests



In [1]:
import molsysmt as msm

In [2]:
molecular_system = msm.build.build_peptide('VPMLK')

In [ ]:
molecular_system = msm.build.add_missing_terminal_cappings(molecular_system, N_terminal='ACE', C_terminal='NME')

In [ ]:
msm.view(molecular_system, standard=True)

In [ ]:
msm.convert(molecular_system, 'V5.pdb')

In [3]:
molsys = msm.systems['chicken villin HP35']['chicken_villin_HP35_solvated.h5msm']

In [4]:
view = msm.view(molsys, viewer='NGLView', standard=True)

In [5]:
comparison = msm.compare(view, molsys, attribute_type='topological', coordinates=True, box=True)

KeyError: "Unable to synchronously open object (object 'id' doesn't exist)"

In [ ]:
comparison

In [ ]:
molsys = msm.systems['chicken villin HP35']['chicken_villin_HP35_solvated.h5msm']
view = msm.view(molsys, viewer='NGLView', standard=True)
comparison = msm.compare(view, molsys, attribute_type='topological', coordinates=True, box=True)

assert comparison
